In [2]:
import numpy as np
import math
from functools import reduce
import time
import sys
import os
import numpy as np

path_project = "\\".join(os.getcwd().split("\\")[:-1])
# caution: path[0] is reserved for script path (or '' in REPL)
sys.path.insert(1, path_project)
from pathlib import Path
import nidaqmx

def lcm(a, b):
    """Calculate the least common multiple of two numbers."""
    return abs(a * b) // math.gcd(a, b)


def lcm_of_list(numbers):
    """Find the LCM of a list of numbers."""
    return reduce(lcm, numbers)

def seqtime(seq_tb):
    return np.sum([pulse[-1] for pulse in seq_tb])

# some constants
Hz = 1e-9 # GHz
kHz = 1e-6 # GHz
MHz = 1e-3 # GHz
pi = np.pi


In [3]:
import nidaqmx
from nidaqmx.constants import TerminalConfiguration, VoltageUnits, Edge, AcquisitionType, READ_ALL_AVAILABLE
from nidaqmx.stream_readers import AnalogSingleChannelReader

from hardware import config as hcf
from hardware.hardwaremanager import HardwareManager
from hardware.pulser.pulser import (
    OutputState,
    TriggerStart,
    TriggerRearm,
    HIGH,
    LOW,
    INF,
    REPEAT_INFINITELY
)
timebase = lcm_of_list(
    [hcf.VDISYN_timebase, hcf.SIDIG_timebase, hcf.PS_timebase, hcf.RSRF_timebase]
)

hm = HardwareManager()
# adds synthesizer, laser, and pulse generator 
hm.add_default_hardware()


VDI Sythesizer Serail Port Open
Connect to Pulse Streamer via JSON-RPC.
IP / Hostname: 192.168.1.26
Pulse Streamer 8/2 firmware: v1.7.2
Client software: v1.7.0
Clock mode set to: 32
1: Internal, 2: Quartz, 3: External, 32: Direct External Sampling


In [4]:
hm.laser.laser_off()
hm.dig.stop_card()
hm.pg.forceFinal()

Card stopped


0

In [5]:
hm.pg.reset()

In [4]:
hm.laser.open()
current_percent = 90
hm.laser.laser_off()
hm.laser.set_analog_control_mode("current")
hm.laser.set_modulation_state("Pulsed")
hm.laser.set_diode_current(current_percent, save_memory=False)

In [5]:
from hardware.daq.sidig import FIFO_DataAcquisition
from hardware.daq.sidig import FIFO_DataAcquisition, DCCOUPLE, TERMIN_INPUT_50OHM, TERMIN_INPUT_1MOHM

SIDIG_ADDRESS = "dev/spcm0"
SIDIG_maxsr = 500e6  # Hz
SIDIG_timebase = int(1 / SIDIG_maxsr * 1e9)  # ns

In [ ]:
num_samples_in_segment = 512*32 
time_off=8*1024*SIDIG_timebase
wait=1e7
time_on = num_samples_in_segment * SIDIG_timebase
seq_dig=[(time_on, HIGH), (time_off, LOW),(wait,LOW)]
seq_laser=[((time_on+time_off)/4, HIGH), ((time_on+time_off)/4, LOW)]*2
seq_laserB=seq_laser+[(wait,LOW)]
hm.pg.setDigital('sdtrig',seq_dig)
hm.pg.setDigital('laser',seq_laser)
hm.pg.setTrigger(start=TriggerStart.SOFTWARE, rearm=TriggerRearm.MANUAL)
hm.pg.plotSeq(plot_all=False)


In [7]:
256* num_samples_in_segment * 64 /1024**3

0.25

In [8]:
type(int(256* num_samples_in_segment * 64))

int

In [9]:
from spcm import units


In [ ]:
# To set the configuration, make a dictionary with the key and value
hm.dig.reset_param()
hm.dig.assign_param(dict(
             readout_ch=hcf.SIDIG_chmap["apd"], 
             amp_input=1000, 
             num_segment=256,
             pretrig_size=16,
             posttrig_size=num_samples_in_segment-16,
             segment_size=num_samples_in_segment,
             terminate_input=TERMIN_INPUT_1MOHM,
             DCCOUPLE = 0,
             sampling_rate=10e6, #hcf.SIDIG_maxsr,
             mem_size= int(256* num_samples_in_segment * 64)*units.Sa
            #  terminate_input=TERMIN_INPUT_50OHM,
             ))


: 

In [ ]:
import time
import numpy as np
import plotly.graph_objects as go
from IPython.display import display, clear_output

# Initialize plot
fig = go.Figure()
fig.add_trace(go.Scatter( y=[], mode="lines", name="Streaming Data"))

start_time = time.time()
run_time = 60*60  # Total script run time
window_size = 20  # Time window to display in seconds

x_data, y_data = [], []

hm.pg.setDigital('sdtrig',seq_dig)

hm.dig.set_config()
hm.pg.stream(n_runs=INF)
hm.dig.start_buffer()
hm.pg.startNow()
hm.laser.laser_on()
while time.time() - start_time < run_time:
    new_y = hm.dig.stream()
    if new_y is None or len(new_y) == 0:
        continue  # Skip empty data
    
    avg = np.mean(new_y)
    print(avg)

    current_time = time.time() - start_time  # Current timestamp relative to start
    x_data.append(current_time)
    y_data.append(avg)

    # Keep only the last 20 seconds of data
    while x_data and (current_time - x_data[0]) > window_size:
        x_data.pop(0)
        y_data.pop(0)

    # Update the figure dynamically
    fig.update_traces( y=y_data)

    clear_output(wait=True)
    display(fig)  # Directly display the figure in Jupyter

    time.sleep(0.1)  # Simulate real-time update delay

print("Streaming Ended!")
hm.laser.laser_off()
hm.dig.stream()
hm.dig.stop_card()

hm.pg.forceFinal()


In [ ]:
hm.laser.laser_off()
hm.dig.stop_card()
hm.pg.forceFinal()

Card stopped


0

In [ ]:
y_data

[]